In [10]:
import os
print( os.environ['PATH'])
import sys

C:\Users\zihao\Anaconda3\Library\bin;C:\Users\zihao\Anaconda3\Library\bin;C:\Users\zihao\Anaconda3\Library\bin;C:\Users\zihao\Anaconda3;C:\Users\zihao\Anaconda3\Library\mingw-w64\bin;C:\Users\zihao\Anaconda3\Library\usr\bin;C:\Users\zihao\Anaconda3\Library\bin;C:\Users\zihao\Anaconda3\Scripts;C:\Users\zihao\Anaconda3\bin;C:\Program Files (x86)\Intel\MPI-RT\4.1.1.036\em64t\bin;C:\Program Files (x86)\Intel\MPI-RT\4.1.1.036\ia32\bin;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\Program Files (x86)\Calibre2\;C:\Program Files\Common Files\Autodesk Shared\;C:\WINDOWS\system32\config\systemprofile\.dnx\bin;C:\Program Files\Microsoft DNX\Dnvm\;C:\Program Files\Microsoft SQL Server\130\Tools\Binn\;C:\Program Files (x86)\Windows Kits\8.1\Windows Performance Toolkit\;C:\Program Files\IBM\ILOG\CPLEX_Studio125\opl\bin\x64_win64;C:\Program Files\IBM\ILOG\CPLEX_Studio125\opl\oplide\;C:\Program Files\IBM\ILOG\CPLEX_Studio125\cplex\bin\x64_win64;

In [11]:
sys.path

['',
 'C:\\Users\\zihao\\Anaconda3\\python36.zip',
 'C:\\Users\\zihao\\Anaconda3\\DLLs',
 'C:\\Users\\zihao\\Anaconda3\\lib',
 'C:\\Users\\zihao\\Anaconda3',
 'C:\\Users\\zihao\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\zihao\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\zihao\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\zihao\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\zihao\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\zihao\\.ipython']

## Question 2: 
Three routes connect an origin and a destination with performance function 
$t_1 =8+0.5X_1 $, $t_2 =1+2X_2$ , and $t_3 =3+0.75X_3$ , with the x’s expressed in thousands of vehicles per hour and the t’s expressed in minutes. If the peak-hour traffic demand is 3000 vehicles, determine user equilibrium traffic flow.

In [1]:
from sympy import *
from sympy.solvers.solveset import linsolve
import numpy as np
# from scipy.linalg import solve

In [2]:
init_printing(use_unicode=False, wrap_line=False) ##To change the printing method of SymPy Live,

In [47]:
def min_UE_x1(euq_t1,euq_t2,total_vol):
    ''' UE with two performace function and total volume '''
    x1 = list(euq_t1.free_symbols)[0]   ## extract variable of 1st performance function
    x2 = list(euq_t2.free_symbols)[0]
    eq_ue = integrate(euq_t1 )+integrate(euq_t2)
    eq_ue_x1 =eq_ue.subs({x2:total_vol-x1})
    eq_1ord= diff(eq_ue_x1,x1,n=1)
    x1_value =solve(eq_1ord,x1)
    x2_value = solve(total_vol-x2-x1_value[0],x2)
    t1 = euq_t1.subs({x1:x1_value[0]})
    t2 = euq_t2.subs({x2:x2_value[0]})
    return x1_value[0],x2_value[0],t1,t2

In [4]:
x1,x2,x3 = symbols(['x1','x2','x3']) 
t1,t2,t3 = symbols(['t1','t2','t3']) 
ls_unk =symbols(['x1','x2','x3']) 
num = len(ls_unk)   ### equals 3
total_vol = 3

#### form an matrix 

In [5]:
# # x1+x2+x3 = 3
t1= 8 + 0.5*x1
t2= 1 + 2*x2
t3= 3 + 0.75*x3

In [6]:
Ax =[x1+x2+x3 - total_vol, t1-t2,t2-t3]
Ax

In [7]:
### create base matrix A and b
element = 0.0
A_COFF = [[element for i in range(num)] for j in range(num)]
A = np.matrix([[element for i in range(num)] for j in range(num)]) ### set  n*n matrix in float 
b = np.array([element for i in range(num)] )

In [8]:
### set value for A and b
for i in range(num):  
    dic = Ax[i].as_coefficients_dict()  ## extract all coeffcient (includes constant 1)
    for j in range(num):  
        A[i,j] = float(dic[ ls_unk[j] ] )
        b[i] = -float(dic[1])

In [9]:
A

matrix([[ 1.  ,  1.  ,  1.  ],
        [ 0.5 , -2.  ,  0.  ],
        [ 0.  ,  2.  , -0.75]])

In [10]:
b

array([ 3., -7.,  2.])

   * using linear solver

In [11]:
np.linalg.solve(A, b)

array([-3.73913043,  2.56521739,  4.17391304])

In [12]:
ls_a = np.linalg.solve(A, b) > np.array([element for i in range(num)])
index = 0
ls_link = [t1,t2,t3]
ls_keep = []
for i in ls_a:
    index+= 1
    if i :
        print ("keep link {}".format(index) ) 
        ls_keep.append(ls_link[index-1])
    else:
        print ("link's volume can not be negative, drop link{}".format(index) )  

link's volume can not be negative, drop link1
keep link 2
keep link 3


In [13]:
ls_keep[0]

user equilibrium (UE)

In [14]:
x2_value_af,x3_value_af,t2_af,t3_af =min_UE_x1(ls_keep[0],ls_keep[1],total_vol)

In [15]:
## validation

if t2_af == t3_af:
    print ('Traffic assignment reacches UE' )
    print ('under 2nd UE, traffic volume of route 1 is {} (1000 vph) '.format( round(x2_value_af,3)) +'\n'+
       'traffic volume of route 2 is {} (1000 vph)'.format( round(x3_value_af,3))
      )
    print ('under 2nd UE, travel time of route 1 is {} minutes'.format( round(t2_af,3)) +'\n'+
       'travel time of route 2 is {} minutes  '.format(round(t3_af,3) )
      )
else:
    print('false')

Traffic assignment reacches UE
under 2nd UE, traffic volume of route 1 is 1.545 (1000 vph) 
traffic volume of route 2 is 1.455 (1000 vph)
under 2nd UE, travel time of route 1 is 4.091 minutes
travel time of route 2 is 4.091 minutes  


## Question 3: 
Two routes connect an origin and destination with performance functions $t_1 =5+3X_1$ and $t_2 =7+X_2$ , the x’s expressed in thousands of vehicles per hour and the t’s expressed in minutes. Total O-D demand is 7000 vehicles in the peak hour. Determine the user equilibrium and system optimal route flows and total travel times.

### a) user equilibrium (UE)

In [16]:
x1,x2,x3 = symbols(['x1','x2','x3']) 
t1,t2,t3 = symbols(['t1','t2','t3']) 

In [17]:
t1 =5+3*x1 
t2 =7+x2
# x2 = 7-x1

In [18]:
## unsing integration
eq_ue = integrate(5+3*x1 )+integrate(7 + x2)

In [19]:
eq_ue_x1 =eq_ue.subs({x2:7-x1})   ## substitue x2 using 7-x1

In [20]:
eq_1ord= diff(eq_ue_x1,x1,n=1)

In [21]:
eq_1ord

In [22]:
x1_value =solve(eq_1ord,x1)

In [23]:
x1_value[0]

In [24]:
x2_value = solve(x1_value[0]+x2-7,x2)

In [25]:
x2_value[0]

In [27]:
print ('under UE, traffic volume of route 1 is {} '.format( float(x1_value[0])) +'\n'+
       'traffic volume of route 2 is {} '.format( float(x2_value[0]))
      )

print ('under UE, travel time of route 1 is {} '.format( float(x1_value[0])*3 + 5) +'\n'+
       'travel time of route 2 is {} '.format( 7 + float(x2_value[0]))
      )

under UE, traffic volume of route 1 is 2.25 
traffic volume of route 2 is 4.75 
under UE, travel time of route 1 is 11.75 
travel time of route 2 is 11.75 


### b) Social optimal  equilibrium (SOE)

In [28]:
X1,X2 = symbols(['X1','X2']) 
T1,T2 = symbols(['T1','T2']) 

In [29]:
## create obj using overall system travel time 
eq_soe = X1*(5+3*X1) + X2*(7 + X2)

In [30]:
eq_soe_X1 =eq_soe.subs({X2:7-X1}) 

In [31]:
eq_soe_X1

In [32]:
eq_1st= diff(eq_soe_X1,X1,n=1)
print (eq_1st)

8*X1 - 16


In [33]:
X1_value =solve(eq_1st,X1)
X2_value = solve(x1_value[0]+X2-7,X2)

In [36]:
print ('under SOE, traffic volume of route 1 is {} '.format( float(X1_value[0])) +'\n'+
       'traffic volume of route 2 is {} '.format( float(X2_value[0]))
      )


print ('under SOE, travel time of route 1 is {} '.format( float(X1_value[0])*3 + 5) +'\n'+
       'travel time of route 2 is {} '.format( 7 + float(X2_value[0]))
      )

under SOE, traffic volume of route 1 is 2.0 
traffic volume of route 2 is 4.75 
under SOE, travel time of route 1 is 11.0 
travel time of route 2 is 11.75 


## Question 4: 

Two routes connect an origin and a destination. The performance functions are $t_1 =3+1.5(X_1 /c_1 )$ and $t_2 =5+4(X_2 /c_2)$, the OD demand is 6000 vehicles per hour, and c1 and c2 are equal to 2000 and 1500 vph, respectively. Proposed capacity improvements will increase c2 by 1000 vph. It is known the current routes are in user equilibrium, and it is estimated that each 1-minute reduction in route travel time will attract an additional 500 vph. What will the user equilibrium flows and total hourly OD demand be after the capacity improvement?

In [37]:
x1,x2,q = symbols(['x1','x2','q']) 
t1,t2,t3 = symbols(['t1','t2','t3']) 
c1=2
c2=1.5
euq_t1 = 3 + 1.5*(x1/c1) 
euq_t2 = 5 + 4*(x2/c2) 
T_vol = 6
sub_x2 = total_vol- x1 

In [44]:
min_UE_x1(euq_t1,euq_t2,T_vol)

In [45]:
x1_value,x2_value,t1,t2 =min_UE_x1(euq_t1,euq_t2,T_vol)

In [48]:
print ('under 1st UE, traffic volume of route 1 is {} (1000 vph) '.format( round(x1_value,3)) +'\n'+
       'traffic volume of route 2 is {} (1000 vph)'.format( round(x2_value,3))
      )

print ('under 1st UE, travel time of route 1 is {} minutes'.format( round(t1,3)) +'\n'+
       'travel time of route 2 is {} minutes  '.format(round(t2,3) )
      )

under 1st UE, traffic volume of route 1 is 5.268 (1000 vph) 
traffic volume of route 2 is 0.732 (1000 vph)
under 1st UE, travel time of route 1 is 6.951 minutes
travel time of route 2 is 6.951 minutes  


In [49]:
### after 
c1=2
c2=2.5
Rate = 0.5 # vph increase per minitue 
euq_t1 = 3 + 1.5*(x1/c1) 
euq_t2 = 5 + 4*(x2/c2) 

In [50]:
x1_value_af,x2_value_af,t1_af,t2_af =min_UE_x1(euq_t1,euq_t2,q)  

## q: OD demand after capacity improvement

In [51]:
t1_af

In [52]:
# q_after = q_before + 2* Rate*  ( t1 -6.951)

In [53]:
solve( 6 + 2*0.5*(6.951-t1_af)- q  ,q)    ### change parameter based on before conditions

In [54]:
q_af = solve( 6 + 2*0.5*(6.951-t1_af)- q  ,q)[0] 
x1_value_af = x1_value_af.subs({q:q_af} )
x2_value_af = x2_value_af.subs({q:q_af} )

t1_af = euq_t1.subs({x1:x1_value_af})
t2_af = euq_t2.subs({x2:x2_value_af})

## validation

if euq_t1.subs({x1:x1_value_af}) == euq_t2.subs({x2:x2_value_af}):
    print ('Traffic assignment reacches UE' )
    print ('under 2nd UE, traffic volume of route 1 is {} (1000 vph) '.format( round(x1_value_af,3)) +'\n'+
       'traffic volume of route 2 is {} (1000 vph)'.format( round(x2_value_af,3))
      )
    print ('under 2nd UE, travel time of route 1 is {} minutes'.format( round(t1_af,3)) +'\n'+
       'travel time of route 2 is {} minutes  '.format(round(t2_af,3) )
      )
else:
    print('false')

Traffic assignment reacches UE
under 2nd UE, traffic volume of route 1 is 5.048 (1000 vph) 
traffic volume of route 2 is 1.116 (1000 vph)
under 2nd UE, travel time of route 1 is 6.786 minutes
travel time of route 2 is 6.786 minutes  
